## Dependencias

In [1]:
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.types import INT,VARCHAR,CHAR,DATETIME,NUMERIC, DECIMAL,TEXT
import json as js
from itertools import repeat

## Motor para SQLite

In [2]:
sqlite = create_engine('sqlite+pysqlite:////home/blaschiquito/Escritorio/entornos/datos/datasets/bd/database.sqlite')


## Motor para MySQL

In [3]:
creds = js.load(open('cred.json','rb'))
mysql = create_engine(f'mysql+pymysql://{creds["user"]}:{creds["password"]}@{creds['host']}/{creds['database']}').connect()

## Listado de tablas

In [4]:
ins = inspect(sqlite)
tablas = ins.get_table_names()
tablas

['Country',
 'League',
 'Match',
 'Player',
 'Player_Attributes',
 'Team',
 'Team_Attributes']

In [5]:
# creamos nuestra conexion con sqlalchemy . connect()

cnx = sqlite.connect()

## Liga/ país

In [6]:
print(tablas[0])
df_pais = pd.read_sql(con=cnx, sql=tablas[0])
df_pais.shape
df_pais.head()

Country


,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy


In [7]:
print(tablas[1])

# leemos la primera tabla, pasandole la conexión y la tabla a seleccionar
df_liga = pd.read_sql(con=cnx, sql=tablas[1])
df_liga.head()

League


,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


In [8]:
# cruzamos la tabla pais con la tabla liga 

df_pais = df_liga.merge(df_pais,left_on='country_id',right_on='id',how='inner')

# quitamos las  columnas 'id_x','id_y' y renombramos la columnas 

df_pais = df_pais.drop(['id_x','id_y'], axis=1).rename(columns={'country_id':'id', 
                                                      'name_x':'nombre_liga',
                                                      'name_y':'nombre_pais'})


# nos muestra como queda el resultado final 

df_pais.head()


,id,nombre_liga,nombre_pais
0,1,Belgium Jupiler League,Belgium
1,1729,England Premier League,England
2,4769,France Ligue 1,France
3,7809,Germany 1. Bundesliga,Germany
4,10257,Italy Serie A,Italy


In [9]:
# mandamos nuestra tabla df_pais a nuestro MySQL workbench ya con su tipo de dato bonito :) 

df_pais.to_sql(con=mysql,index=False,name='tabla_liga', if_exists='replace',dtype={'id':INT,
                                                                                   'nombre_liga':VARCHAR(24),
                                                                                   'nombre_pais':VARCHAR(11)})

11

## Partido/ Apuestas

In [10]:
print(tablas[2])
df_partido = pd.read_sql(con=cnx,sql=tablas[2])
df_partido.head()


Match


,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [11]:
df_partido.drop(['country_id']+[f'home_player_X{i+1}' for i in range(11)]+\
                [f'home_player_Y{i+1}' for i in range(11)]+\
                [f'away_player_X{i+1}' for i in range(11)]+\
                [f'away_player_Y{i+1}' for i in range(11)],
                axis=1,inplace=True)


In [12]:
apuestas = df_partido.columns.to_list()[-30:]
apuestas


['B365H',
 'B365D',
 'B365A',
 'BWH',
 'BWD',
 'BWA',
 'IWH',
 'IWD',
 'IWA',
 'LBH',
 'LBD',
 'LBA',
 'PSH',
 'PSD',
 'PSA',
 'WHH',
 'WHD',
 'WHA',
 'SJH',
 'SJD',
 'SJA',
 'VCH',
 'VCD',
 'VCA',
 'GBH',
 'GBD',
 'GBA',
 'BSH',
 'BSD',
 'BSA']

In [13]:
df_apuestas = df_partido[['id']+apuestas].copy()
df_apuestas.head()

,id,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [14]:
d = {'id':INT}
d.update(dict(zip(apuestas,repeat(DECIMAL(10,2)))))
d

{'id': sqlalchemy.sql.sqltypes.INTEGER,
 'B365H': DECIMAL(precision=10, scale=2),
 'B365D': DECIMAL(precision=10, scale=2),
 'B365A': DECIMAL(precision=10, scale=2),
 'BWH': DECIMAL(precision=10, scale=2),
 'BWD': DECIMAL(precision=10, scale=2),
 'BWA': DECIMAL(precision=10, scale=2),
 'IWH': DECIMAL(precision=10, scale=2),
 'IWD': DECIMAL(precision=10, scale=2),
 'IWA': DECIMAL(precision=10, scale=2),
 'LBH': DECIMAL(precision=10, scale=2),
 'LBD': DECIMAL(precision=10, scale=2),
 'LBA': DECIMAL(precision=10, scale=2),
 'PSH': DECIMAL(precision=10, scale=2),
 'PSD': DECIMAL(precision=10, scale=2),
 'PSA': DECIMAL(precision=10, scale=2),
 'WHH': DECIMAL(precision=10, scale=2),
 'WHD': DECIMAL(precision=10, scale=2),
 'WHA': DECIMAL(precision=10, scale=2),
 'SJH': DECIMAL(precision=10, scale=2),
 'SJD': DECIMAL(precision=10, scale=2),
 'SJA': DECIMAL(precision=10, scale=2),
 'VCH': DECIMAL(precision=10, scale=2),
 'VCD': DECIMAL(precision=10, scale=2),
 'VCA': DECIMAL(precision=10, scal

In [15]:
df_apuestas.head()

,id,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,...,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,...,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,...,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,...,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,...,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [16]:
df_apuestas.to_sql(con=mysql,name='tabla_apuestas',if_exists='replace',index=False,dtype=d)

25979

## Tio gustavo

In [ ]:
# tio gustavo 


# 
# tio gustavo 

print(tablas[0])
df_pais = pd.read_sql(con=cnx,sql=tablas[0])
df_pais.shape
df_pais.head()
#-------------*****************-

# tio gustavo 
df_pais = df_liga.merge(df_pais,left_on='country_id',right_on='id',how='inner')
df_pais = df_pais.drop(['id_x','id_y'],axis=1).rename(columns={'country_id':'id',
                                                     'name_x':'nombre_liga',
                                                     'name_y':'nombre_pais'})
df_pais.head()

#---------------*

print(tablas[2])
df_partido = pd.read_sql(con=cnx,sql=tablas[2])

# -----------------*******************----------


#-------------------

# tio gustavo

df_pais.to_sql(con=mysql,index=False,name='tbl_liga',if_exists='replace',dtype={'id':INT,
                                                               'nombre_liga':VARCHAR(24),
                                                               'nombre_pais':VARCHAR(11)})


#---------*************************************
df_partido.drop(['country_id']+[f'home_player_X{i+1}' for i in range(11)]+\
                [f'home_player_Y{i+1}' for i in range(11)]+\
                [f'away_player_X{i+1}' for i in range(11)]+\
                [f'away_player_Y{i+1}' for i in range(11)],
                axis=1,inplace=True)


# --------------------****************------------

apuestas = df_partido.columns.to_list()[-30:]
apuestas

# ----------**********--------

df_apuestas = df_partido[['id']+apuestas].copy()

# --------*/************---------

d = {'id':INT}
d.update(dict(zip(apuestas,repeat(DECIMAL(10,2)))))
d

# ------------*********--------
df_apuestas.to_sql(con=mysql,name='tbl_apuestas',dtype=d,if_exists='replace',index=False)

#------------*************-------
df_partido.drop(apuestas,axis=1,inplace=True)

#------------------***********----------

xml = ['goal',
'shoton',
'shotoff',
'foulcommit',
'card',
'cross',
'corner',
'possession']

# ---------************---------------

d = {'id':INT,
 'season':CHAR(9),
     'stage':INT,
     'league_id':INT,
     'date':DATETIME,
 'match_api_id':INT,
 'home_team_api_id':INT,
 'away_team_api_id':INT,
 'home_team_goal':INT,
 'away_team_goal':INT
}
d.update(dict(zip([f'home_player_{i+1}' for i in range(11)]+[f'away_player_{i+1}' for i in range(11)],repeat(INT))))

#----------*************--------------

d = {'id':INT}
d.update(dict(zip(xml,repeat(TEXT))))

In [17]:
df_partido.drop(apuestas,axis=1,inplace=True)


In [18]:
df_partido.columns

Index(['id', 'league_id', 'season', 'stage', 'date', 'match_api_id',
       'home_team_api_id', 'away_team_api_id', 'home_team_goal',
       'away_team_goal', 'home_player_1', 'home_player_2', 'home_player_3',
       'home_player_4', 'home_player_5', 'home_player_6', 'home_player_7',
       'home_player_8', 'home_player_9', 'home_player_10', 'home_player_11',
       'away_player_1', 'away_player_2', 'away_player_3', 'away_player_4',
       'away_player_5', 'away_player_6', 'away_player_7', 'away_player_8',
       'away_player_9', 'away_player_10', 'away_player_11', 'goal', 'shoton',
       'shotoff', 'foulcommit', 'card', 'cross', 'corner', 'possession'],
      dtype='object')

In [19]:
# creamos una lista con los archivos xml del DataFrame df_partido
xml = ['goal', 
       'shoton',
       'shotoff', 
       'foulcommit',
        'card',
        'cross',
        'corner', 
        'possession']

In [20]:
df_xml = df_partido[['id']+xml].copy()

df_xml.shape
df_xml.head()

,id,goal,shoton,shotoff,foulcommit,card,cross,corner,possession
0,1,None,None,None,None,None,None,None,None
1,2,None,None,None,None,None,None,None,None
2,3,None,None,None,None,None,None,None,None
3,4,None,None,None,None,None,None,None,None
4,5,None,None,None,None,None,None,None,None


In [21]:
df_partido.drop(xml,axis=1,inplace=True)


In [22]:
d = {'id':INT,
     'season': CHAR(9),
     'stage':INT,
     'league_id':INT,
    'date':DATETIME,
    'match_api_id':INT,
    'home_team_api_id':INT, 
    'away_team_api_id':INT,
    'home_team_goal':INT,
    'away_team_goal':INT
    }

d.update(dict(zip([f'home_player_{i+1}' for i in range(11)]+[f'away_player_{i+1}' for i in range(11)],repeat(INT))))

In [23]:
df_partido.to_sql(con=mysql,name='tabla_partido',if_exists='replace',index=False,dtype=d)

25979

In [24]:
d= {'id':INT}
d.update(dict(zip(xml,repeat(TEXT))))

In [25]:
df_xml.to_sql(con=mysql,name='tabla_xml',if_exists='replace',index=False,dtype=d)

25979